## Importieren von *Instagram*-Daten, die mit *zeeschuimer* gesammelt wurden

In diesem Notebook schauen wir uns an, wie wir mit *Instagram*-Daten, die mit dem Tool [*zeeschuimer*](https://github.com/digitalmethodsinitiative/zeeschuimer) gesammelt wurden, importieren und überprüfen können. Wir schauen uns zwei Varianten an: <br> 1) Import einer `csv`-Datei, die mit dem Tool [*4CAT*](https://4cat.nl/) generiert wurde (Daten wurden zuvor mit *zeeschuimer* erhoben und dann in *4CAT* importiert) <br> 2) Import einer [`ndjson`](https://en.wikipedia.org/wiki/JSON_streaming)-Datei, die direkt aus *zeeschuimer* exportiert wurde

Für beide Varianten nutzen wir [`R`](https://www.r-project.org/)-Code.

### Allgemeiner Hinweise

Informationen zur Installation von *4CAT* gibt es im [Wiki zum Tool auf GitHub](https://github.com/digitalmethodsinitiative/4cat/wiki/Installing-4CAT) sowie im [Tutorial-Video auf *YouTube*](https://www.youtube.com/watch?v=oWsB7bvNfOY&list=PLWukutaRyIn31H0uPfkYlmbWvo83PnXXo&index=3). Anweisungen für die Installation von *zeeschuimer* gibt es im [GitHub Repository des Tools](https://github.com/digitalmethodsinitiative/zeeschuimer).

Anleitungen zur Nutzung von *zeeschuimer* und *4CAT* sowie Anwendungsbeispiele gibt es im [*Worksheet - Capturing social media data with Zeeschuimer and 4CAT*](https://docs.google.com/document/d/19MAiqX7Vx1FcNJ44K-vSdnKDVC5gcFwtrSQiewnwW8A/edit?tab=t.0#heading=h.3sfjpc6v42ag) sowie im Dokument [*4CAT Exercises*](https://docs.google.com/document/d/1po-sOB8tDRZlvWrEayu97cGh_qsBuW0URd4md0_tv7k/edit?tab=t.0).

### Import aus `csv`-Datei

Im nachfolgenden Beispiel nutzen wir eine Funktion aus dem `R`-Paket [`readr`](https://readr.tidyverse.org/), um eine mit *4CAT* aus mit *zeeschuimer* gesammelten Daten generierte `csv`-Datei zu importieren.

Um die Datei in diesem Notebook importieren zu können, muss diese zunächst in die Online-Umgebung hochgeladen werden. Dies geht über den File Explorer (Upload File) auf der linken Seite des *Jupyter Lab*-Interface.

In [ ]:
library(readr)

In [ ]:
insta1 <- read_csv("INSERT_FILE_NAME_HERE") # Namen der entsprechenden Datei (inkl. Dateiendung) einfügen

In [ ]:
names(insta1)

In [ ]:
library(dplyr)

Für GenZ `R` User gibt es als Alternative zum Paket `dplyr` auch das Paket `genzplyr`: https://hadley.github.io/genzplyr/ <br> ;-)

In [ ]:
glimpse(insta1)

### Import aus `ndjson`-Datei

Hier schauen wir uns zwei Varianten mit zwei unterschiedlichen `R`-Paketen an: [`jsonlite`](https://github.com/jeroen/jsonlite) und [`ndjson`](https://github.com/hrbrmstr/ndjson)

Auch hier muss die aus *zeeschuimer* exportierte Datei wie im Falle der `csv`-Datei erst einmal in die Online-Umgebung hochgeladen werden (s.o.).

In [ ]:
library(jsonlite)

In [ ]:
insta2 <- stream_in(file("INSERT_FILE_NAME_HERE"))

In [ ]:
names(insta2)

In [ ]:
glimpse(insta2)

Wir sehen hier, dass die Funktion aus dem Paket `jsonlite` standardmäßig nur die oberste Ebene aus der `ndjson`-Datei in einen `R` `data.frame` überführt. Dieser hat entsprechend nur 9 Variablen. Die Variable `data` ist selbst wiederum ein eigener `data.frame`. Zur Erinnerung: Daten bzw. Key-Value-Pairs in `json`-Dateien sind geschachtelt.

In der zweiten Variante nutzen wir Funktionen aus den beiden `R`-Paketen `ndjson` und `dplyr`, um die Daten aus der `ndjson`-Datei vollständig in eine sogenannte flache Struktur (zweidimensionaler `data.frame`) zu überführen.

In [ ]:
library(ndjson)

In [ ]:
data_list <- ndjson::stream_in("INSERT_FILE_NAME_HERE") # Namen der entsprechenden Datei (inkl. Dateiendung) einfügen

In [ ]:
flat_tibble <- dplyr::bind_rows(data_list)

In [ ]:
names(flat_tibble)

In [ ]:
glimpse(flat_tibble)

Wir sehen, dass wir nun 1476 Variablen haben. Alle zuvor geschachelten Variablen sind nun auf derselben Ebene in einem zweidimensionalen `data.frame`. Das ist natürlich unübersichtlich und doe große Mehrheit der 1476 Variablen sind für uns nicht von Interesse.

### Parser

Um die Daten aus der `ndjson`-Datei in ein Format zu überführen, mit dem wir arbeiten können, müssen wir diese "parsen". Zu diesem Zweck habe ich mit Unterstützung von [*Google Gemini*](https://gemini.google.com/app) einen experimentellen Parser geschrieben, der sich am Output aus der `csv`-Konvertierungsfunktion von *4CAT* orientiert.

**NB**: Die Funktionen des Parsers sind sehr experimentell und noch nicht ausführlich getestet. Sie dienen daher nur der Anschauung in diesem Beispiel und sollten nicht ohne weitere Überprüfung für die tatsächliche Datenaufbereitung genutzt werden.

Um die Funktionen nutzen zu können, müssen wir diese definieren. Dies geschieht im separaten `R`-Skript namens `parse_insta.R`.

In [ ]:
source("parse_insta.R")

In [ ]:
insta_parsed <- parse_instagram_posts("INSERT_FILE_NAME_HERE") # Namen der entsprechenden Datei (inkl. Dateiendung) einfügen

In [ ]:
names(insta_parsed)

In [ ]:
glimpse(insta_parsed)

Den resultierenden `data.frame` können wir nun auch als `csv`-Datei exportieren.

In [ ]:
write_csv(insta_parsed, "./insta_parsed.csv")

Die `csv`-Datei können wir dann auch wieder auf unseren lokalen Rechner herunterladen: Rechtsklick auf den Dateinamen im File Explorer links und dann "Download" auswählen.